In [196]:
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Intrinsic;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Math;
open Microsoft.Quantum.Convert;
open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Preparation;
open Microsoft.Quantum.Characterization;
open Microsoft.Quantum.Oracles;

In [298]:
// oracle f(x) = 1 if x = 00..00, 11..11
operation Oracle(inputQubits : Qubit[], output : Qubit) : Unit is Adj+Ctl {
    Controlled X(inputQubits, output);
    within {
        for input in inputQubits {
            X(input);
        }
    }
    apply {
        Controlled X(inputQubits, output);
    }
}

Oracle

In [299]:
// Grover diffusion operator
operation GroverDiffusion(inputQubits : Qubit[]) : Unit is Adj+Ctl {

    within {
        ApplyToEachA(H, inputQubits);
        ApplyToEachA(X, inputQubits);
    } apply {
        Controlled Z(Most(inputQubits), Tail(inputQubits));
    }
}

GroverDiffusion

In [300]:
operation GroverPow(power: Int, inputQubits : Qubit[], output : Qubit): Unit is Adj+Ctl {
    for i in 1 .. power {
        Oracle(inputQubits, output);
        GroverDiffusion(inputQubits);
    }
}

GroverPow

In [301]:
operation QuantumCounting(inputQubits : Qubit[], output : Qubit, targetQubits : Qubit[]) : Unit {
    ApplyToEachA(H, inputQubits);
    H(output);
    ApplyToEachA(H, targetQubits);
    for i in 0 .. Length(targetQubits) - 1 {
        Controlled GroverPow([targetQubits[Length(targetQubits) - i - 1]], (2 ^ i, inputQubits, output));
    }
    Adjoint QFT(BigEndian(targetQubits));
}

QuantumCounting

In [302]:
open Microsoft.Quantum.Extensions.Convert;
operation MeasureTarget() : Int {
    use inputQubits = Qubit[4];
    use output = Qubit();
    use targetQubits = Qubit[5];
    
    QuantumCounting(inputQubits, output, targetQubits);
    
    let a = BoolArrayAsInt(Reversed(ResultArrayAsBoolArray(MultiM(targetQubits))));
    ResetAll(inputQubits);
    Reset(output);
    ResetAll(targetQubits);
    
    let theta = 2.0 * IntAsDouble(a) * PI() / IntAsDouble(2 ^ Length(targetQubits));
    let numSol= (1.0 - PowD(Sin(theta / 2.0), 2.0)) * IntAsDouble(2 ^ (Length(inputQubits) + 1));
    let x = Round(numSol);
    Message($"numSolutions: {x}");
    return a;
}

MeasureTarget

In [303]:
%simulate MeasureTarget

numSolutions: 0


16